# Ejemplos con Hive

## Descripción de las variables

El dataset, obtenido de <a target = "_blank" href="https://www.transtats.bts.gov/Fields.asp?Table_ID=236">este link</a> está compuesto por las siguientes variables referidas siempre al año 2018:

1. **Month** 1-4
2. **DayofMonth** 1-31
3. **DayOfWeek** 1 (Monday) - 7 (Sunday)
4. **FlightDate** fecha del vuelo
5. **Origin** código IATA del aeropuerto de origen
6. **OriginCity** ciudad donde está el aeropuerto de origen
7. **Dest** código IATA del aeropuerto de destino
8. **DestCity** ciudad donde está el aeropuerto de destino  
9. **DepTime** hora real de salida (local, hhmm)
10. **DepDelay** retraso a la salida, en minutos
11. **ArrTime** hora real de llegada (local, hhmm)
12. **ArrDelay** retraso a la llegada, en minutos: se considera que un vuelo ha llegado "on time" si aterrizó menos de 15 minutos más tarde de la hora prevista en el Computerized Reservations Systems (CRS).
13. **Cancelled** si el vuelo fue cancelado (1 = sí, 0 = no)
14. **CancellationCode** razón de cancelación (A = aparato, B = tiempo atmosférico, C = NAS, D = seguridad)
15. **Diverted** si el vuelo ha sido desviado (1 = sí, 0 = no)
16. **ActualElapsedTime** tiempo real invertido en el vuelo
17. **AirTime** en minutos
18. **Distance** en millas
19. **CarrierDelay** en minutos: El retraso del transportista está bajo el control del transportista aéreo. Ejemplos de sucesos que pueden determinar el retraso del transportista son: limpieza de la aeronave, daño de la aeronave, espera de la llegada de los pasajeros o la tripulación de conexión, equipaje, impacto de un pájaro, carga de equipaje, servicio de comidas, computadora, equipo del transportista, problemas legales de la tripulación (descanso del piloto o acompañante) , daños por mercancías peligrosas, inspección de ingeniería, abastecimiento de combustible, pasajeros discapacitados, tripulación retrasada, servicio de inodoros, mantenimiento, ventas excesivas, servicio de agua potable, denegación de viaje a pasajeros en mal estado, proceso de embarque muy lento, equipaje de mano no válido, retrasos de peso y equilibrio.
20. **WeatherDelay** en minutos: causado por condiciones atmosféricas extremas o peligrosas, previstas o que se han manifestado antes del despegue, durante el viaje, o a la llegada.
21. **NASDelay** en minutos: retraso causado por el National Airspace System (NAS) por motivos como condiciones meteorológicas (perjudiciales pero no extremas), operaciones del aeropuerto, mucho tráfico aéreo, problemas con los controladores aéreos, etc.
22. **SecurityDelay** en minutos: causado por la evacuación de una terminal, re-embarque de un avión debido a brechas en la seguridad, fallos en dispositivos del control de seguridad, colas demasiado largas en el control de seguridad, etc.
23. **LateAircraftDelay** en minutos: debido al propio retraso del avión al llegar, problemas para conseguir aterrizar en un aeropuerto a una hora más tardía de la que estaba prevista.

Leemos el fichero CSV utilizando el delimitador por defecto de Spark (","). La primera línea contiene encabezados (nombres de columnas) por lo que no es parte de los datos y debemos indicarlo con la opción header.

## 1. Leemos los datos desde el CSV que hay en Google Cloud Storage

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession\
    .builder\
    .appName("EJhive")\
    .getOrCreate()

In [2]:
# Esto no hace nada: la lectura es lazy así que no se lee en realidad hasta que ejecutemos una acción sobre flightsDF
# Solamente se comprueba que exista el fichero en esa ruta, y se leen los nombres de columnas
flightsDF = spark.read.option("header", "true")\
                 .csv("flights-jan-apr-2018.csv")

## 2. Vamos a mostrar el contenido del metastore de Hive, que ahora mismo no tiene nada

In [3]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



## 3. Creamos una vista temporal de un DF que tiene solo dos columnas

Esto solamente añade metadatos al metastore de Hive, que además se borrarán cuando cerremos el notebook. No guarda datos físicos de la tabla en ningún lado, puesto que el DF está en memoria. O mejor dicho, el DF del que proviene esta vista temporal "no está en ningún lado", porque no hemos cacheado weatherDistanceDF así que cualquier consulta SQL que hagamos sobre la tabla `weatherDistanceTable` provoca que se tenga que re-calcular el DF `weatherDistanceDF` sobre el cual está creada dicha tabla temporal.

In [4]:
weatherDistanceDF = flightsDF.select("WeatherDelay", "Distance")
weatherDistanceDF.createOrReplaceTempView("weatherDistanceTable")

Ahora vemos que la tabla se ha creado como tabla temporal

In [5]:
spark.sql("show tables").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|        |weatherdistancetable|       true|
+--------+--------------------+-----------+



Podemos hacer consultas sobre ella

In [6]:
spark.sql("select * from weatherDistanceTable limit 5").show()

+------------+--------+
|WeatherDelay|Distance|
+------------+--------+
|        null|  374.00|
|        null|  198.00|
|        0.00|  198.00|
|        null|  198.00|
|        null|  198.00|
+------------+--------+



## 4. Creamos una tabla persistente manejada, guardando como tabla el resultado de una operación con el DF

In [7]:
flightsJFK = flightsDF.where("Origin = 'JFK'")
flightsJFK.write.saveAsTable("flightsjfk") # es una acción: se guardan físicamente los datos en algún sitio de HDFS

Si volvemos a mostrar las tablas que existen, veremos la nueva. Vemos que **no** es temporal, pero no sabemos si es manejada o es externa.

In [8]:
spark.sql("show tables").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|          flightsjfk|      false|
|        |weatherdistancetable|       true|
+--------+--------------------+-----------+



### Para saber si una tabla es temporal, o bien es externa o es manejada

In [9]:
spark.sql("describe formatted flightsjfk").show(50)

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|               Month|              string|   null|
|          DayofMonth|              string|   null|
|           DayOfWeek|              string|   null|
|          FlightDate|              string|   null|
|              Origin|              string|   null|
|          OriginCity|              string|   null|
|                Dest|              string|   null|
|            DestCity|              string|   null|
|             DepTime|              string|   null|
|            DepDelay|              string|   null|
|             ArrTime|              string|   null|
|            ArrDelay|              string|   null|
|           Cancelled|              string|   null|
|    CancellationCode|              string|   null|
|            Diverted|              string|   null|
|   ActualElapsedTime|              string|   null|
|           

In [10]:
spark.sql("describe formatted flightsjfk").where("col_name = 'Type'").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|    Type|  MANAGED|       |
+--------+---------+-------+



## 5. Guardamos flightsDF como fichero parquet en HDFS (nada de tablas aún)

In [11]:
flightsSFO = flightsDF.where("Origin = 'SFO'")\
                      .select("FlightDate", "Origin", "Dest", "Distance")
flightsSFO.write.mode("overwrite").parquet("/flightsSFO.parquet")

In [14]:
!hdfs dfs -ls /flightsSFO.parquet

El sistema no puede encontrar la ruta especificada.
"-classpath" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


## 6. Ahora vamos a crear una tabla EXTERNA a partir del fichero /flightsSFO.parquet

Ojo: tenemos que especificar bien el esquema de la tabla que estamos creando a partir del fichero. El DF que hemos guardado en Parquet tenía 4 columnas, todas de tipo string

In [11]:
flightsSFO.printSchema()

root
 |-- FlightDate: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: string (nullable = true)



In [13]:
spark.sql("create external table flightsSFO(FlightDate string, Origin string, Dest string, Distance string)\
          stored as parquet location '/flightsSFO.parquet'")

DataFrame[]

### Comprobamos que ahora tenemos una tabla más, persistente, llamada flightSFO

In [14]:
spark.sql("show tables").show(truncate = False)

+--------+--------------------+-----------+
|database|tableName           |isTemporary|
+--------+--------------------+-----------+
|default |flightsjfk          |false      |
|default |flightssfo          |false      |
|        |weatherdistancetable|true       |
+--------+--------------------+-----------+



### ¿La tabla flightsSFO es externa, o es manejada?

In [15]:
spark.sql("describe formatted flightssfo").show(50)

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|          FlightDate|              string|   null|
|              Origin|              string|   null|
|                Dest|              string|   null|
|            Distance|              string|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|            Database|             default|       |
|               Table|          flightssfo|       |
|               Owner|                root|       |
|        Created Time|Sat Jun 27 11:08:...|       |
|         Last Access|Thu Jan 01 00:00:...|       |
|          Created By|         Spark 2.3.4|       |
|                Type|            EXTERNAL|       |
|            Provider|                hive|       |
|    Table Properties|[transient_lastDd...|       |
|          Statistics|         67093 bytes|       |
|           

In [16]:
spark.sql("describe formatted flightssfo").where("col_name = 'Type'").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|    Type| EXTERNAL|       |
+--------+---------+-------+



## 7. Borramos tabla externa y comprobamos que el fichero Parquet sigue ahí

In [17]:
spark.sql("drop table flightssfo")
spark.sql("show tables").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| default|          flightsjfk|      false|
|        |weatherdistancetable|       true|
+--------+--------------------+-----------+



### Comprobemos que sigue existiendo la carpeta /flightsSFO.parquet

In [18]:
!hdfs dfs -ls /flightsSFO.parquet

Found 4 items
-rw-r--r--   2 root hadoop          0 2020-06-27 10:58 /flightsSFO.parquet/_SUCCESS
-rw-r--r--   2 root hadoop      23854 2020-06-27 10:58 /flightsSFO.parquet/part-00000-b35f850f-3f08-414a-b167-e6d68e7870a3-c000.snappy.parquet
-rw-r--r--   2 root hadoop      29340 2020-06-27 10:58 /flightsSFO.parquet/part-00001-b35f850f-3f08-414a-b167-e6d68e7870a3-c000.snappy.parquet
-rw-r--r--   2 root hadoop      13899 2020-06-27 10:58 /flightsSFO.parquet/part-00002-b35f850f-3f08-414a-b167-e6d68e7870a3-c000.snappy.parquet


## 8. Comprobamos dónde están los datos de la tabla persistente manejada que habíamos guardado con saveAsTable

In [26]:
!hdfs dfs -ls /user/hive/warehouse/flightsjfk

Found 4 items
-rw-r--r--   2 root hadoop          0 2020-06-27 10:52 /user/hive/warehouse/flightsjfk/_SUCCESS
-rw-r--r--   2 root hadoop     177679 2020-06-27 10:52 /user/hive/warehouse/flightsjfk/part-00000-0378514d-d352-4ded-8397-f6b21a3228a2-c000.snappy.parquet
-rw-r--r--   2 root hadoop     245568 2020-06-27 10:52 /user/hive/warehouse/flightsjfk/part-00001-0378514d-d352-4ded-8397-f6b21a3228a2-c000.snappy.parquet
-rw-r--r--   2 root hadoop      71420 2020-06-27 10:52 /user/hive/warehouse/flightsjfk/part-00002-0378514d-d352-4ded-8397-f6b21a3228a2-c000.snappy.parquet


## 9. Borramos la tabla y comprobamos que, al ser manejada, Spark ha borrado físicamente esos datos

In [27]:
spark.sql("drop table flightsjfk")
spark.sql("show tables").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|        |weatherdistancetable|       true|
+--------+--------------------+-----------+



In [31]:
!hdfs dfs -ls /user/hive/warehouse/flightsjfk

ls: `/user/hive/warehouse/flightsjfk': No such file or directory
